In [ ]:
import cv2
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
# Object Counting
# question a)

In [ ]:
# Open the video file
vid_obj = cv2.VideoCapture('DatasetC.mpg')

# Read frames
ret, reference_frame = vid_obj.read()
ret, frame1 = vid_obj.read()
for _ in range(98):  # Skipping 98 frames to get to the 100th frame
    ret, _ = vid_obj.read()
ret, frame2 = vid_obj.read()

# Convert frames to RGB (assuming they are in BGR format)
reference_frame = cv2.cvtColor(reference_frame, cv2.COLOR_BGR2RGB)
frame1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2RGB) #frame1
frame2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2RGB) #frame100

In [ ]:
def ICV_rgb2grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

In [ ]:
def ICV_TemporalDifference(reference_frame, frame, threshold):
    if reference_frame.shape != frame.shape:
        raise ValueError("Input images must have the same shape.")
        
    frame_difference = np.abs(reference_frame.astype(np.int32) - frame.astype(np.int32))
    threshold_result = (frame_difference > threshold).astype(np.uint8) * 255
    return frame_difference, threshold_result

In [ ]:
plt.figure(1)
plt.imshow(reference_frame)
plt.axis('off')
plt.savefig('referenceFrame.png')

In [ ]:
plt.figure(2)
plt.imshow(frame1)
plt.axis('off')
plt.savefig('frame1.png')

In [ ]:
plt.figure(3)
plt.imshow(frame2)
plt.axis('off')
plt.savefig('frame100.png')

In [ ]:
Threshold = 20

ReferenceFrame=ICV_rgb2grayscale(reference_frame)
GrayFrame1=ICV_rgb2grayscale(frame1)
GrayFrame2=ICV_rgb2grayscale(frame2)

frame_difference1, threshold_result1 = ICV_TemporalDifference(ReferenceFrame, GrayFrame1, Threshold)
frame_difference2, threshold_result2 = ICV_TemporalDifference(ReferenceFrame, GrayFrame2, Threshold)
plt.figure(4)
plt.imshow(frame_difference1, cmap='gray')
plt.axis('off')
plt.savefig('FrameDifferencing1.png')

In [ ]:
plt.figure(5)
plt.imshow(threshold_result1, cmap='gray')
plt.axis('off')
plt.savefig('ThresholdResult1.png')

In [ ]:
plt.figure(6)
plt.imshow(frame_difference2, cmap='gray')
plt.axis('off')
plt.savefig('FrameDifferencing2.png')

In [ ]:
plt.figure(7)
plt.imshow(threshold_result2, cmap='gray')
plt.axis('off')
plt.savefig('ThresholdResult2.png')

In [ ]:
# question b)

In [ ]:
# Open the video file
def ICV_sequenceFrameDiff():
    vid_obj_b = cv2.VideoCapture('DatasetC.mpg')
    
    # Read the first frame
    ret, reference_frame_b = vid_obj_b.read()
    
    reference_frame_b = cv2.cvtColor(reference_frame_b, cv2.COLOR_BGR2RGB)
    
    # Loop through the frames
    for iFrame in range(2, int(vid_obj_b.get(cv2.CAP_PROP_FRAME_COUNT))):
        print(f'Frame #{iFrame} | Reference Frame #{iFrame-1}')
    
        # Read current frame
        ret, current_frame = vid_obj_b.read()
    
        # Check if the frame is read successfully
        if not ret:
            print(f"Error: Could not read frame {iFrame}.")
            break
    
        current_frame = cv2.cvtColor(current_frame, cv2.COLOR_BGR2RGB)
    
    
        gray_frame1=ICV_rgb2grayscale(reference_frame_b)
        gray_frame2=ICV_rgb2grayscale(current_frame)
    
    
        # Display the current frame
        plt.figure(8)
        plt.imshow(current_frame)
        plt.axis('off')
        plt.title(f'Reference Frame #{iFrame-1} |Frame #{iFrame}')
    
        # # Calculate frame differencing and threshold
        threshold_value = 20
        frame_differencing, threshold_result = ICV_TemporalDifference(gray_frame1, gray_frame2, threshold_value)
    
        # # Display frame differencing
        plt.figure(9)
        plt.imshow(frame_differencing, cmap='gray')
        plt.axis('off')
        plt.title(f'Reference Frame #{iFrame-1} |Frame #{iFrame}')
    
        # # Display threshold result
        plt.figure(10)
        plt.imshow(threshold_result, cmap='gray')
        plt.axis('off')
        plt.title(f'Reference Frame #{iFrame-1} |Frame #{iFrame}')
    
        reference_frame_b = current_frame
        
        # # Show the figures
        plt.show()
    # Release the video capture object
    vid_obj_b.release()

ICV_sequenceFrameDiff()

In [ ]:
# question c)

In [ ]:
vid_obj_c = cv2.VideoCapture('DatasetC.mpg')

def ICV_generateBackground(vid_object):
    # Open the video file
    
    threshold = 20
    
    # Read the first frame
    ret, reference_frame_c = vid_object.read()
    
    reference_frame_c = cv2.cvtColor(reference_frame_c, cv2.COLOR_BGR2RGB)
    
    # Get image dimensions
    H, W, _ = reference_frame_c.shape
    
    num_pixel = np.zeros((H, W), dtype=int)
    background = np.zeros((H, W, 3), dtype=float)
    image = np.zeros((H, W, 3), dtype=float)
    
    # Loop through the frames
    for iFrame in range(2, int(vid_object.get(cv2.CAP_PROP_FRAME_COUNT))):
    
        # Read frames
        ret, currentFrame = vid_object.read()
    
        # Check if the frame is read successfully
        if not ret:
            print(f"Error: Could not read frame {iFrame}.")
            break
    
        grayFrame1 = ICV_rgb2grayscale(reference_frame_c)
        grayFrame2 = ICV_rgb2grayscale(currentFrame)
    
        # Calculate frame differencing and threshold
        _, threshold_result = ICV_TemporalDifference(grayFrame1, grayFrame2, threshold)
    
        # Update pixel count and accumulate pixel values
        for i in range(H):
            for j in range(W):
                if threshold_result[i, j] == 0:
                    num_pixel[i, j] += 1
                    image[i, j, :] += currentFrame[i, j, :]
                    
        reference_frame_c=currentFrame
    
    
    # Calculate the background
    for i in range(H):
        for j in range(W):
            if num_pixel[i, j] != 0:
                background[i, j, :] = image[i, j, :] / num_pixel[i, j]
    
    background = np.uint8(background)
    
    # Display the background
    plt.imshow(cv2.cvtColor(background, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()
    
    # Release the video capture object
    vid_object.release()
    
    return background

background = ICV_generateBackground(vid_obj_c)

In [ ]:
# Question d)

In [ ]:
def find_contours(binary_diff):
    contours = []
    height, width = binary_diff.shape[:2]
    visited = np.zeros_like(binary_diff)

    def dfs(x, y, current_contour):
        stack = [(x, y)]
        while stack:
            cx, cy = stack.pop()
            if (
                cx < 0
                or cy < 0
                or cx >= height
                or cy >= width
                or binary_diff[cx, cy] == 0
                or visited[cx, cy] == 1
            ):
                continue
            visited[cx, cy] = 1
            current_contour.append((cx, cy))
            for dx in range(-1, 2):
                for dy in range(-1, 2):
                    stack.append((cx + dx, cy + dy))

    for i in range(height):
        for j in range(width):
            if binary_diff[i, j] == 255 and visited[i, j] == 0:
                current_contour = []
                dfs(i, j, current_contour)
                if current_contour:
                    # Calculate the center of the contours
                    contours_center = (
                        sum([x for x, y in current_contour]) // len(current_contour),
                        sum([y for x, y in current_contour]) // len(current_contour),
                    )
                    current_contour.append(contours_center)
                    contours.append(current_contour)

    return contours

def ICV_countObjects(binary_diff, min_contour_area=100):
    contours = find_contours(binary_diff)
    # Count contours with area greater than a threshold (min_contour_area)
    num_objects = sum(cv2.contourArea(np.array(contour)) > min_contour_area for contour in contours if len(contour) > 2)

    return num_objects


def ICV_countObjectVideo(videoFrames, threshold):
    videoFrames = np.array(videoFrames)
    numberOfObjects = []
    
    # Use the first frame as the background
    background = ICV_rgb2grayscale(videoFrames[0])
    
    # Count the number based on different frames
    for i in range(1, len(videoFrames)):
        _, threshold_result = ICV_TemporalDifference(background, ICV_rgb2grayscale(videoFrames[i]), threshold)
        num_objects = ICV_countObjects(threshold_result)
        numberOfObjects.append(num_objects)

    return numberOfObjects

# Open the video file
vid_object = cv2.VideoCapture('DatasetC.mpg')

# Check if the video file is opened successfully
if not vid_object.isOpened():
    print("Error: Could not open the video file.")
    exit()

# Read the video frames
vid_frames = []
while True:
    ret, frame = vid_object.read()
    if not ret:
        break
    vid_frames.append(frame)

# Close the video file
vid_object.release()

# Count the number of objects in each frame
num_objects_list = ICV_countObjectVideo(vid_frames, 20)

# Plot the results
frame_numbers = np.arange(2, len(num_objects_list) + 2)  # Adjusted frame numbers (starting from the 2nd frame)
plt.bar(frame_numbers, num_objects_list)
plt.xlabel('Frame Number')
plt.ylabel('Number of Objects')
plt.title('Number of Moving Objects in Each Frame')
plt.show()